In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
def Number_page():
    url = "https://www.newegg.com/p/pl?d=xiaomi+12"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")
    page_text = doc.find(class_="list-tool-pagination-text")
    num_page = int(str(page_text).split("strong")[1].split(">")[-1][0])
    return num_page

number_pages = Number_page()
number_pages

8

In [4]:
def Extract(page):
    url_page = f"https://www.newegg.com/p/pl?d=xiaomi+12&page={page}"
    doc_soup = BeautifulSoup(requests.get(url_page).text, "html.parser")
    return doc_soup

In [5]:
def TransormDataFrame():
    columns = ["description", "price", "link"]
    items_found = {column : [] for column in columns}
    for page in range(1, number_pages):
        doc_page = Extract(page)
        div_container= doc_page.find(class_="item-cells-wrap border-cells items-grid-view four-cells expulsion-one-cell")
        items = div_container.find_all(string=re.compile("Xiaomi"))
        
        for item in items:
            list_item = []
            parent = item.parent
            if parent.name != "a":
                continue
            link = parent["href"]
            next_parent = parent.parent.parent
            class_price = next_parent.find(class_="price-current")
            if class_price.strong == None :
                continue
            try :
                price = int(class_price.strong.string)
            except :
                price = int(class_price.strong.string.replace(",", ""))
            finally :
                pass
            list_item.append(item)
            list_item.append(price)
            list_item.append(link)
            for column, row_item in zip(items_found.keys(),list_item) :
                items_found[column].append(row_item)
    return pd.DataFrame(items_found).sort_values(by="price").set_index("description")
    

In [6]:
df = TransormDataFrame()

In [7]:
df.head()

,price,link
description,,
Xiaomi Redmi A2 Dual-SIM 32GB ROM + 2GB RAM (Only GSM | No CDMA) Factory Unlocked 4G/LTE Smartphone (Light Blue) - International Version,79,https://www.newegg.com/p/23B-00KF-00010
Xiaomi Redmi 9A Dual-SIM 32GB ROM + 2GB RAM (GSM Only | No CDMA) Factory Unlocked 4G/LTE Smartphone (Peacock Green) - International Version,101,https://www.newegg.com/xiaomi-6-53-gsm-hspa-lt...
Xiaomi Redmi 10 2022 Dual-SIM 64GB ROM + 4GB RAM (GSM | CDMA) Factory Unlocked 4G/LTE Smartphone (Carbon Gray) - International Version,109,https://www.newegg.com/p/23B-0016-00AJ1
"Xiaomi Redmi 9A - Smartphone 2 GB + 32 GB, Dual Sim, Grigio (Granite Grey)",128,https://www.newegg.com/p/23B-0016-008K1
"Xiaomi Redmi 9A 6.53? HD+ Dot Drop Display, Dual SIM GSM Unlocked US and Global 4G LTE (Global Version) (Ocean Green, 32GB) (MZB9961EU)",136,https://www.newegg.com/p/23B-0016-00705


In [10]:
df.shape

(248, 2)

In [38]:
print(df.link[1])

https://www.newegg.com/xiaomi-6-53-gsm-hspa-lte-green/p/23B-0016-00782


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, Xiaomi Redmi A2 Dual-SIM 32GB ROM + 2GB RAM (Only GSM | No CDMA) Factory Unlocked 4G/LTE Smartphone (Light Blue) - International Version to Xiaomi Mi 11 Ultra 6.81-inch 3200x1440P AMOLED Display 5G Smartphone 8GB 256GB 5000mAh Battery MIUI 12.5 - White
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   price   248 non-null    int64 
 1   link    248 non-null    object
dtypes: int64(1), object(1)
memory usage: 13.9+ KB


In [18]:
df_Under300 = df[df.price <300]
print(df_Under300.shape)

(29, 2)


In [23]:
df_Under300

,price,link
description,,
Xiaomi Redmi A2 Dual-SIM 32GB ROM + 2GB RAM (Only GSM | No CDMA) Factory Unlocked 4G/LTE Smartphone (Light Blue) - International Version,79,https://www.newegg.com/p/23B-00KF-00010
Xiaomi Redmi 9A Dual-SIM 32GB ROM + 2GB RAM (GSM Only | No CDMA) Factory Unlocked 4G/LTE Smartphone (Peacock Green) - International Version,101,https://www.newegg.com/xiaomi-6-53-gsm-hspa-lt...
Xiaomi Redmi 10 2022 Dual-SIM 64GB ROM + 4GB RAM (GSM | CDMA) Factory Unlocked 4G/LTE Smartphone (Carbon Gray) - International Version,109,https://www.newegg.com/p/23B-0016-00AJ1
"Xiaomi Redmi 9A - Smartphone 2 GB + 32 GB, Dual Sim, Grigio (Granite Grey)",128,https://www.newegg.com/p/23B-0016-008K1
"Xiaomi Redmi 9A 6.53? HD+ Dot Drop Display, Dual SIM GSM Unlocked US and Global 4G LTE (Global Version) (Ocean Green, 32GB) (MZB9961EU)",136,https://www.newegg.com/p/23B-0016-00705
Xiaomi Redmi Note 7 Dual-SIM 64GB ROM + 4GB RAM (GSM Only | No CDMA) Factory Unlocked 4G/LTE Smartphone (Neptune Blue) - International Version,159,https://www.newegg.com/xiaomi-6-3-gsm-hspa-lte...
"Xiaomi Redmi A1 4G 32GB + 2GB Global Version Factory Unlocked 6.52"" 8MP Dual Camera (Not USA Market) + (w/Fast Car Charger Bundle + 64GB SD) (Black)",169,https://www.newegg.com/p/23B-00KF-00003
Xiaomi Redmi Note 11S Dual-SIM 128GB ROM + 6GB RAM (GSM Only | No CDMA) Factory Unlocked 4G/LTE Smartphone (Graphite Gray) - International Version,198,https://www.newegg.com/xiaomi-6-4-gsm-hspa-lte...
Xiaomi Redmi A2 Dual-SIM 32GB ROM + 2GB RAM (Only GSM | No CDMA) Factory Unlocked 4G/LTE Smartphone (Light Green) - International Version,225,https://www.newegg.com/p/23B-00KF-00011


In [22]:
print(df_Under300.link[27])

https://www.newegg.com/p/23B-0016-00A35
